# 심리상담 챗봇 만들기 

In [52]:
import pandas as pd
import numpy as np
import sys
import os
from matplotlib import colors
import random
import pickle 
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
import tensorflow as tf
from tensorflow.keras import models

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.21.5
TensorFlow Version :2.10.1
Matplotlib Version :3.5.3


In [ ]:
plt.rc("font", family='Malgun Gothic')
plt.rc('axes', unicode_minus = False)

In [54]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510458 -0.7733839   0.5927711  ...  0.57923526  0.3268346
  -0.65089655]
 [ 0.03518831 -0.06713754  0.03035226 ...  0.12334981  0.53281206
  -0.31036612]]


In [58]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/kairess/mental-health-chatbot/master/wellness_dataset_original.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')

In [59]:
df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


In [60]:
df1 = df.iloc[:,[1,2]]
df1 = df1.dropna(subset='챗봇')
df1

,유저,챗봇
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.
...,...,...
5196,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?
5197,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.
5213,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.
5214,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?


In [61]:
df1.isnull().sum()

유저    0
챗봇    0
dtype: int64

In [62]:
df1.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [63]:
df1['embedding'] = df1['유저'].map(lambda x: list(model.encode(x)))
df1.head()

,유저,챗봇,embedding
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401..."
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949..."
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792..."
15,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045..."
16,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624..."


In [71]:
df2 = df2.iloc[:,[0,1]]
df2.columns = ['유저', '챗봇']
df2

,유저,챗봇
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.


In [72]:
df2.isnull().sum()

유저    0
챗봇    0
dtype: int64

In [73]:
df2['embedding'] = df2['유저'].map(lambda x: list(model.encode(x)))
df2.head()

,유저,챗봇,embedding
0,12시 땡!,하루가 또 가네요.,"[-0.19072711, 0.32147592, 0.59873784, 0.446178..."
1,1지망 학교 떨어졌어,위로해 드립니다.,"[0.123206064, 0.64236826, 0.68536025, -0.42912..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,"[0.024357582, -0.22393535, 0.22917657, -0.4501..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,"[0.03628536, -0.21048243, 0.07269376, -0.54036..."
4,PPL 심하네,눈살이 찌푸려지죠.,"[0.02203077, 0.70552164, -0.17157197, 0.165971..."


In [74]:
chat_df = pd.concat([df1,df2])

In [75]:
chat_df

,유저,챗봇,embedding
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401..."
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949..."
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792..."
15,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045..."
16,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624..."
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,"[0.16655466, -0.17954743, 0.24957272, -0.46702..."
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,"[0.16655466, -0.17954743, 0.24957272, -0.46702..."
11820,흑기사 해주는 짝남.,설렜겠어요.,"[-0.35615, -0.11077429, 0.33425468, 0.97426826..."
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,"[-0.84649175, -0.46591726, -0.041984845, 0.032..."


## 문장 유사도  구하기

In [92]:
text = '친구가 날 버린 거 같아서 너무 힘들다'
text2 = '내가 바보같아'
text3 = '세상이 날 버린 거 같아'

embedding = model.encode(text3)

In [93]:
chat_df['distance'] = chat_df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
chat_df.head()

,유저,챗봇,embedding,distance
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401...",0.361785
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949...",0.440894
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792...",0.255487
15,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045...",0.388310
16,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624...",0.331457


In [101]:
answer = chat_df.loc[chat_df['distance'].idxmax()]
answer

유저                                                나 버림 받은 거 같아
챗봇                                                      아닐거예요.
embedding    [-0.32598445, -0.3941628, 0.05194258, -0.25523...
distance                                              0.760436
Name: 499, dtype: object

In [107]:
answer2 = chat_df.nlargest(3, 'distance')
answer2

,유저,챗봇,embedding,distance
499,나 버림 받은 거 같아,아닐거예요.,"[-0.32598445, -0.3941628, 0.05194258, -0.25523...",0.760436
7342,이 세상 하직할뻔 했네.,여기서 끝내기엔 당신은 소중한 사람입니다.,"[-0.15761814, -0.42287314, 0.6470201, -0.35752...",0.680678
168,나만 어디 버려져 있는 느낌이에요.,제가 당신의 이야기를 들을게요. 무슨 일이 있으셨나요?,"[0.051334683, -0.70041955, 0.16410798, -0.1926...",0.671416


In [112]:
print('유사한 질문', ':', answer['유저'])
print('챗봇 답변', ':', answer2['챗봇'].values[:])
print('유사도', ':', answer['distance'])

유사한 질문 : 나 버림 받은 거 같아
챗봇 답변 : ['아닐거예요.' '여기서 끝내기엔 당신은 소중한 사람입니다.' '제가 당신의 이야기를 들을게요. 무슨 일이 있으셨나요?']
유사도 : 0.7604362368583679
